# Machine Learning API using Flask

### Building the API

In [17]:
import os
import pandas as pd

In [3]:
ml_api_script_file = os.path.join(os.path.pardir, 'Titanic-dataset-experimentation', 'Models', 'ml_api.py')

In [22]:
%%writefile $ml_api_script_file

from flask import Flask, request
import pandas as pd
import numpy as np
import json
import pickle
import os

app = Flask(__name__)

# Load model and scaler files
models_folder = os.path.join(os.path.pardir, 'Titanic-dataset-experimentation', 'Models')
model_filepath = os.path.join(models_folder, 'LR_model.pkl')
scaler_filepath = os.path.join(models_folder, 'LR_scaler.pkl')

model = pickle.load(open('LR_model.pkl', 'rb'))
scaler = pickle.load(open('LR_scaler.pkl', 'rb'))

columns = [ u'Age', u'Fare', u'FamilySize', \
           u'IsMother', u'IsMale', u'Deck_A', u'Deck_B', u'Deck_C', u'Deck_D', \
           u'Deck_E', u'Deck_F', u'Deck_G', u'Deck_Z', u'Pclass_1', u'Pclass_2', \
           u'Pclass_3', u'Title_Lady', u'Title_Master', u'Title_Miss', u'Title_Mr', \
           u'Title_Mrs', u'Title_Officer', u'Title_Sir', u'Fare_Bin_very_low', \
           u'Fare_Bin_low', u'Fare_Bin_high', u'Fare_Bin_very_high', u'Embarked_C', \
           u'Embarked_Q', u'Embarked_S', u'AgeState_Adult', u'AgeState_Child']

@app.route('/api', methods=['POST'])
def make_predictions():
    data = json.dumps(request.get_json(force=True)) # Read json object and convert to json string
    df = pd.read_json(data) # Create dataframe using the json string data
    passenger_ids = df['PassengerId'].ravel() # PassengerId, flattened into a vector
    actuals = df['Survived'].ravel()
    X = df[columns].as_matrix().astype('float') # Extract required columns and convert to matrix
    X_scaled = scaler.transform(X)
    predictions = model.predict(X_scaled)
    df_response = pd.DataFrame({'PassengerId' : passenger_ids, 'Prediction' : predictions, 'Actual' : actuals}) # Response data frame sent by the API
    
    return df_response.to_json()

if __name__ == '__main__':
    # Hosting the flask app
    app.run(port=10001, debug=True)
    

Overwriting ..\Titanic-dataset-experimentation\Models\ml_api.py


### Invoking the API

In [18]:
train_df = pd.read_csv('train_processed.csv')

survived_passengers = train_df[train_df['Survived'] == 1][:5]

In [19]:
survived_passengers

,Survived,Age,Fare,PassengerId,FamilySize,IsMother,IsMale,Deck_A,Deck_B,Deck_C,...,Title_Sir,Fare_Bin_very_low,Fare_Bin_low,Fare_Bin_high,Fare_Bin_very_high,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child
1,1,38.0,71.2833,2,2,0,0,0,0,1,...,0,0,0,0,1,1,0,0,1,0
2,1,26.0,7.9250,3,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
3,1,35.0,53.1000,4,2,0,0,0,0,1,...,0,0,0,0,1,0,0,1,1,0
8,1,27.0,11.1333,9,3,1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
9,1,14.0,30.0708,10,2,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1


In [20]:
import requests

def make_api_request(data):
    url = 'http://127.0.0.1:10001/api'
    # Make POST request
    r = requests.post(url, data)
    
    return r.json()

In [23]:
make_api_request(survived_passengers.to_json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [24]:
# Comparing results
result = make_api_request(train_df.to_json())
df_result = pd.read_json(json.dumps(result))
df_result.head()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [25]:
# accuracy
import numpy as np
np.mean(df_result.Actual == df_result.Predicted)

NameError: name 'df_result' is not defined